In [33]:
library(tidyverse)
library(gbm)
install.packages("caret")
library(caret)
library(pROC)
install.packages("kernlab")
library(kernlab)

In [99]:
ATAC_pred <-  readRDS(snakemake@input[["input_predictions"]])
head(ATAC_pred)

ERROR while rich displaying an object: Error in sprintf(wrap, header, body): 'fmt' length exceeds maximal format length 8192

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. repr::mime2repr[[mime]](obj)
8. 

,sample,chr10_400,chr10_40,chr10_4,chr10_401,chr10_402,chr10_403,chr10_404,chr10_405,chr10_406,...,chr9_39,chr9_391,chr9_392,chr9_393,chr9_394,chr9_395,chr9_396,chr9_397,chr9_398,chr9_399
V1,PGDX16568P,0.7230317,-0.7588268,0.4208489,-0.6723128,-0.5630451,-0.3621892,0.1080097,0.5673606,0.8912879,...,-0.28893444,0.4172587,-0.35850255,-0.59667008,0.213152421,-0.05634686,1.0313072,0.9238163,0.7802505,-1.241104e+00
V2,PGDX16569P,0.7399565,-0.7760577,0.6783101,-1.0239559,-0.9394263,-0.4478646,-0.2877922,0.6070161,0.5563960,...,-0.88173916,0.5727677,0.04509077,-0.18199032,0.223162314,0.01890688,0.7499369,0.5836826,1.9460280,3.885927e-05
V3,PGDX16570P,0.3496038,-1.0549674,1.0595707,-1.0136229,-0.2704319,-0.7518339,-0.4827620,1.1331901,0.4683745,...,-0.87749692,-0.1190669,-0.16284899,-0.33915476,-0.758361774,0.12763408,0.6870010,0.1656184,1.2600567,-3.909340e-01
V4,PGDX16571P,0.6926064,-1.0619340,0.7160739,-0.9855300,-0.7274114,-0.6534164,-0.1886758,0.9001139,0.4322670,...,0.08429159,0.2364777,0.19323870,0.08986791,0.005394444,0.35687186,0.6942288,1.0012787,1.2281786,5.488598e-01
V5,PGDX16579P,0.5513661,-0.8115573,0.7259174,-1.0018507,-1.4139831,-1.1265568,0.1282141,0.1948417,0.4402955,...,-0.03577759,-0.2025408,0.02771539,-0.56592734,-0.728716827,-0.30322679,1.3999326,0.8260956,0.7130110,-1.686135e-01
V6,PGDX16582P,0.6371399,-0.8106510,0.9486726,-0.7565931,-0.4783383,-0.6145264,-0.1753195,1.1650789,0.8071104,...,-0.43223861,1.0109715,0.37269361,-0.21003093,0.204732985,0.33833870,1.1724080,0.8483280,0.7592924,-4.978362e-01


In [100]:
#ATAC_pred <- ATAC_pred[1:3000, 1:200]

In [101]:
#input_sample_type <- as.character(snakemake@input[["input_sample_type"]])
sample_types <- readRDS(file = snakemake@input[["input_sample_types"]])
colnames(sample_types) <- c("sample", "sample_type")
head(sample_types)

sample,sample_type
PGDX10344P1,Gastric_cancer
PGDX10346P1,Gastric_cancer
PGDX10349P1,Gastric_cancer
PGDX10351P1,Gastric_cancer
PGDX10566P,Ovarian_Cancer
PGDX10567P,Ovarian_Cancer


In [103]:
data <- merge(ATAC_pred, sample_types, by="sample")
data <- data %>% mutate(sample_type01 = ifelse(sample_type == "Healthy", "Healthy", "Cancer"))
head(data)

sample,chr10_400,chr10_40,chr10_4,chr10_401,chr10_402,chr10_403,chr10_404,chr10_405,chr10_406,...,chr10_87,chr10_88,chr10_89,chr10_90,chr10_9,chr10_91,chr10_92,chr10_93,sample_type,sample_type01
PGDX10344P1,0.461834968,-0.7921477,1.3681636,-0.7163189,-0.6608585,-0.5016680,0.27747952,0.2684491,0.2367779,...,0.6421282,-0.36206803,-0.13753231,0.5191983,-1.4227223,0.6661580,0.57377837,-0.567893307,Gastric_cancer,Cancer
PGDX10346P1,1.304813218,-0.8418882,0.8443271,-0.4941999,-0.8788231,-0.5331765,-0.05029568,0.9772446,0.3863285,...,0.1260843,-0.28380232,0.34693146,0.2904037,-0.8783722,1.5355984,1.18787626,-1.008745353,Gastric_cancer,Cancer
PGDX10349P1,0.008374035,-1.0373401,0.3171460,-1.1464607,-0.7755127,-0.8866752,-0.14621765,0.4629197,2.4528914,...,1.8635021,0.67147159,-0.34641505,0.4879573,-1.1607931,0.8601661,-0.09532705,-0.448417202,Gastric_cancer,Cancer
PGDX10351P1,1.458763719,-1.2382937,0.4583231,-1.0579030,-0.6479878,-0.2323826,-0.18474186,0.9527618,0.4454378,...,0.9012095,0.41911290,-0.33487548,0.6607363,-1.0572646,1.4480447,0.19680994,0.001857877,Gastric_cancer,Cancer
PGDX10566P,0.434508778,-0.9720612,1.0997871,-0.9459747,-0.3650844,-0.7671493,-0.28449386,0.8190828,0.7041095,...,0.5782184,0.04131913,-0.03821612,0.8999317,-0.6865825,1.7660091,0.84234866,-0.118967138,Ovarian_Cancer,Cancer
PGDX10567P,1.123182664,-0.5442369,1.2666051,-0.1795052,-0.8291937,-0.7764733,0.17056626,0.6018786,0.7284151,...,2.1866379,1.42259571,-0.68138372,0.6014320,-0.8126152,1.7296002,-0.31842339,-0.037113319,Ovarian_Cancer,Cancer


In [104]:
data <- data %>% select(-sample_type)

In [105]:
data <- data %>% select(-sample)
head(data)

chr10_400,chr10_40,chr10_4,chr10_401,chr10_402,chr10_403,chr10_404,chr10_405,chr10_406,chr10_407,...,chr10_86,chr10_87,chr10_88,chr10_89,chr10_90,chr10_9,chr10_91,chr10_92,chr10_93,sample_type01
0.461834968,-0.7921477,1.3681636,-0.7163189,-0.6608585,-0.5016680,0.27747952,0.2684491,0.2367779,1.89577717,...,0.003825133,0.6421282,-0.36206803,-0.13753231,0.5191983,-1.4227223,0.6661580,0.57377837,-0.567893307,Cancer
1.304813218,-0.8418882,0.8443271,-0.4941999,-0.8788231,-0.5331765,-0.05029568,0.9772446,0.3863285,0.04332981,...,-0.165975169,0.1260843,-0.28380232,0.34693146,0.2904037,-0.8783722,1.5355984,1.18787626,-1.008745353,Cancer
0.008374035,-1.0373401,0.3171460,-1.1464607,-0.7755127,-0.8866752,-0.14621765,0.4629197,2.4528914,1.26556128,...,-0.522081335,1.8635021,0.67147159,-0.34641505,0.4879573,-1.1607931,0.8601661,-0.09532705,-0.448417202,Cancer
1.458763719,-1.2382937,0.4583231,-1.0579030,-0.6479878,-0.2323826,-0.18474186,0.9527618,0.4454378,0.89176900,...,-0.195148585,0.9012095,0.41911290,-0.33487548,0.6607363,-1.0572646,1.4480447,0.19680994,0.001857877,Cancer
0.434508778,-0.9720612,1.0997871,-0.9459747,-0.3650844,-0.7671493,-0.28449386,0.8190828,0.7041095,0.89757640,...,-0.270333541,0.5782184,0.04131913,-0.03821612,0.8999317,-0.6865825,1.7660091,0.84234866,-0.118967138,Cancer
1.123182664,-0.5442369,1.2666051,-0.1795052,-0.8291937,-0.7764733,0.17056626,0.6018786,0.7284151,0.51709329,...,-0.986101393,2.1866379,1.42259571,-0.68138372,0.6014320,-0.8126152,1.7296002,-0.31842339,-0.037113319,Cancer


In [108]:
str(data$sample_type01)

 Factor w/ 2 levels "Cancer","Healthy": 1 1 1 1 1 1 1 1 1 1 ...


In [107]:
data$sample_type01 <- as.factor(data$sample_type01)

In [109]:
levels(data$sample_type01)

[1] "Cancer"  "Healthy"

In [146]:
cross_validation <- function(dataset, k_inner_cv, k_outer_cv){
    
    observed  <- dataset$sample_type01
   
    return_tibble <- tibble(observed = observed)
    
    for (i in 1:k_outer_cv){ # repeated Cross-validation loop
    
    set.seed(i)
    cvfolds <- cut(seq_len(nrow(dataset)), breaks = k_inner_cv, labels = F)
    cvfolds <- sample(cvfolds)

    predicted <- rep(NA, nrow(dataset))
    
        for (n in 1:k_inner_cv){
            rows      <- which(cvfolds==n)
            testdata  <- dataset[rows,]
            testlabels <- testdata$sample_type01
            testdata <- testdata %>% select(-sample_type01)
            
            traindata <- dataset
         
            traindata <- dataset[-rows,]
           
            trainlabels <- traindata$sample_type01
            #traindata <- traindata %>% select(-sample_type01)

 
            ################# Nested cross validation #######################
            set.seed(0)
            seeds <- vector(mode = "list", length = 11)
            for(i in 1:10) seeds[[i]]<- sample.int(n=1000, 11)
            #for the last model
            seeds[[11]]<-sample.int(1000, 1)

            trControl_svm <- trainControl(method = "repeatedcv", 
                                      seeds = seeds,
                                      number = 10, 
                                      repeats = 1, 
                                      classProbs = TRUE,
                                      verboseIter = ifelse(is.null(getOption('knitr.in.progress')), TRUE, FALSE))

            svmGrid <- expand.grid(C = c(0.01, 0.1, 0.5, 1, 2, 3, 4, 5, 6, 7))

            fit <- train(sample_type01 ~ .,
                         data = traindata, 
                         method = "svmLinear",
                         tuneGrid = svmGrid,
                         trControl = trControl_svm,
                         preProc = c("center", "scale"),
                         verbose=F)
            
            message(plot(fit))
            message("besttune C")
            message(fit$bestTune$C)
            message("fit")
            message(fit)
            #################################################################
            #message(fit)

            fitControl <- trainControl(classProbs = TRUE)
            fit2 <- train(sample_type01 ~ .,
                         data = traindata,
                         method =  "svmLinear",
                         trControl = fitControl,
                         verbose = FALSE,
                         tuneGrid = data.frame(C = fit$bestTune$C),
                        preProc = c("center", "scale"))
            predicted[rows] <- predict(fit2, newdata = testdata, type = "prob")[,2]
        
    }
    current_round_tibble <- tibble(predicted = predicted)
    return_tibble <- cbind(return_tibble, current_round_tibble)
    } # end of outer cv loop
    
    return(return_tibble)
}

In [147]:
results <- cross_validation(data, k_inner_cv = 10, k_outer_cv = 10)

+ Fold01.Rep1: C= 1 
- Fold01.Rep1: C= 1 
+ Fold01.Rep1: C= 2 
- Fold01.Rep1: C= 2 
+ Fold01.Rep1: C= 3 
- Fold01.Rep1: C= 3 
+ Fold01.Rep1: C= 4 
- Fold01.Rep1: C= 4 
+ Fold01.Rep1: C= 5 
- Fold01.Rep1: C= 5 
+ Fold01.Rep1: C= 6 
- Fold01.Rep1: C= 6 
+ Fold01.Rep1: C= 7 
- Fold01.Rep1: C= 7 
+ Fold01.Rep1: C= 8 
- Fold01.Rep1: C= 8 
+ Fold01.Rep1: C= 9 
- Fold01.Rep1: C= 9 
+ Fold01.Rep1: C=10 
- Fold01.Rep1: C=10 
+ Fold02.Rep1: C= 1 
- Fold02.Rep1: C= 1 
+ Fold02.Rep1: C= 2 
- Fold02.Rep1: C= 2 
+ Fold02.Rep1: C= 3 
- Fold02.Rep1: C= 3 
+ Fold02.Rep1: C= 4 
- Fold02.Rep1: C= 4 
+ Fold02.Rep1: C= 5 
- Fold02.Rep1: C= 5 
+ Fold02.Rep1: C= 6 
- Fold02.Rep1: C= 6 
+ Fold02.Rep1: C= 7 
- Fold02.Rep1: C= 7 
+ Fold02.Rep1: C= 8 
- Fold02.Rep1: C= 8 
+ Fold02.Rep1: C= 9 
- Fold02.Rep1: C= 9 
+ Fold02.Rep1: C=10 
- Fold02.Rep1: C=10 
+ Fold03.Rep1: C= 1 
- Fold03.Rep1: C= 1 
+ Fold03.Rep1: C= 2 
- Fold03.Rep1: C= 2 
+ Fold03.Rep1: C= 3 
- Fold03.Rep1: C= 3 
+ Fold03.Rep1: C= 4 
- Fold03.Rep1

In [148]:
head(results)

observed,prob,prob
Cancer,0.4223283,0.6536439
Cancer,0.5517766,0.5437114
Cancer,0.2152466,0.2478794
Cancer,0.4805241,0.5753186
Cancer,0.5091694,0.5429416
Cancer,0.4599332,0.3944589


In [149]:
results <- tibble(results, .name_repair = "unique")
head(results)

New names:
* prob -> prob...2
* prob -> prob...3


observed,prob...2,prob...3
Cancer,0.4223283,0.6536439
Cancer,0.5517766,0.5437114
Cancer,0.2152466,0.2478794
Cancer,0.4805241,0.5753186
Cancer,0.5091694,0.5429416
Cancer,0.4599332,0.3944589


In [ ]:
saveRDS(results, file = snakemake@output[["SVM_output"]])

In [ ]:
# Create multiple curves to plot
rocs <- roc(observed ~ predicted...2 + 
                    predicted...3 + 
                    predicted...4 + 
                    predicted...5 + 
                    predicted...6 +
                    predicted...7 +
                    predicted...8 + 
                    predicted...9 + 
                    predicted...10 + 
                    predicted...11, data = results)
ggroc(rocs)

In [ ]:
roc1 <- roc(results$observed, results$predicted...2)
roc2 <- roc(results$observed, results$predicted...3)
roc3 <- roc(results$observed, results$predicted...4)
roc4 <- roc(results$observed, results$predicted...5)
roc5 <- roc(results$observed, results$predicted...6)
roc6 <- roc(results$observed, results$predicted...7)
roc7 <- roc(results$observed, results$predicted...8)
roc8 <- roc(results$observed, results$predicted...9)
roc9 <- roc(results$observed, results$predicted...10)
roc10 <- roc(results$observed, results$predicted...11)

In [ ]:
paste("Mean AUC over 10 repetitions of 10-fold CV: ", mean(c(auc(roc1), auc(roc2), auc(roc3), auc(roc4), auc(roc5), auc(roc6), auc(roc7), auc(roc8), auc(roc9), auc(roc10))), sep = "")

In [ ]:
results_01 <- results %>% mutate(predicted...2_01 = ifelse(predicted...2 > 0.5, 1, 0), 
                                 predicted...3_01 = ifelse(predicted...3 > 0.5, 1, 0), 
                                 predicted...4_01 = ifelse(predicted...4 > 0.5, 1, 0),
                                 predicted...5_01 = ifelse(predicted...5 > 0.5, 1, 0),
                                 predicted...6_01 = ifelse(predicted...6 > 0.5, 1, 0),
                                 predicted...7_01 = ifelse(predicted...7 > 0.5, 1, 0),
                                 predicted...8_01 = ifelse(predicted...8 > 0.5, 1, 0),
                                 predicted...9_01 = ifelse(predicted...9 > 0.5, 1, 0),
                                 predicted...10_01 = ifelse(predicted...10 > 0.5, 1, 0),
                                 predicted...11_01 = ifelse(predicted...11 > 0.5, 1, 0))

In [ ]:
error_rates <- c(mean(results_01$observed != results_01$predicted...2_01),
               mean(results_01$observed != results_01$predicted...3_01), 
               mean(results_01$observed != results_01$predicted...4_01),
               mean(results_01$observed != results_01$predicted...5_01),
               mean(results_01$observed != results_01$predicted...6_01),
               mean(results_01$observed != results_01$predicted...7_01),
               mean(results_01$observed != results_01$predicted...8_01),
               mean(results_01$observed != results_01$predicted...9_01), 
               mean(results_01$observed != results_01$predicted...10_01), 
               mean(results_01$observed != results_01$predicted...11_01))

error_rates

paste("Mean error rate over 10 repetitions of 10-fold CV: ", mean(error_rates), sep = "")

In [ ]:
accuracies <- c(mean(results_01$observed == results_01$predicted...2_01),
               mean(results_01$observed == results_01$predicted...3_01), 
               mean(results_01$observed == results_01$predicted...4_01),
               mean(results_01$observed == results_01$predicted...5_01),
               mean(results_01$observed == results_01$predicted...6_01),
               mean(results_01$observed == results_01$predicted...7_01),
               mean(results_01$observed == results_01$predicted...8_01),
               mean(results_01$observed == results_01$predicted...9_01), 
               mean(results_01$observed == results_01$predicted...10_01), 
               mean(results_01$observed == results_01$predicted...11_01))

accuracies
paste("Mean accuracy over 10 repetitions of 10-fold CV: ", mean(accuracies), sep = "")